In [10]:
import torch
import torchvision
from tqdm import tqdm
from torchvision import datasets
from torch.utils.data import DataLoader
from torch import nn
from torchvision import transforms

In [11]:
training=datasets.CIFAR100(train=True,root="data",download=True,transform=transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
]))
train_dataloader=DataLoader(training,batch_size=64)

In [13]:
VGG19=[64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']

In [14]:
class VGG(nn.Module):
    def __init__(self,in_channel=3,num_classes=100):
        super(VGG,self).__init__()
        self.in_channel=in_channel
        self.NN=self.conv_layers(VGG19,in_channel)
        self.classifier = nn.Sequential(
            nn.Linear(512*7*7, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes)
            )
        self.flatten=nn.Flatten(start_dim=1)
    def forward(self,x):
        x=self.NN(x)
        x=self.flatten(x)
        x=self.classifier(x)
        return x


    def conv_layers(self, architecture,in_channel):
        layers = []
        
        for x in architecture:
            if type(x) == int:
                out_channels = x
                
                layers += [nn.Conv2d(in_channels=in_channel,out_channels=out_channels,
                                     kernel_size=3, stride=1, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU()]
                in_channel = x
            elif x == 'M':
                layers += [nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))]
                
        return nn.Sequential(*layers)

In [15]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [16]:
model=VGG().to(device)
learning_rate=0.001

In [17]:
loss_fun=nn.CrossEntropyLoss()
optimiser=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [18]:
data,target=next(iter(train_dataloader))

In [19]:
def accuracy_check(x,y,model):
    model.eval()
    x=x.to(device=device)
    y=y.to(device=device)
    with torch.no_grad():
            scores=model(x)
            scores=nn.functional.softmax(scores,dim=1)
            _,predictions=scores.max(1)
            correct=(predictions==y).sum()
            total= predictions.size(0)
    print(float(correct/total))

In [20]:
data=data.to(device)
target=target.to(device)
for i in (range(200)):
        scores=model(data)
        loss=loss_fun(scores,target)
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        accuracy_check(data,target,model)

In [21]:
print(loss_fun(scores,target))